# JDBC Predicate Pushdown

In [3]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Predicate Pushdown") \
    .config('spark.jars.packages', 'org.xerial:sqlite-jdbc:3.39.3.0') \
    .master("local[*]") \
    .getOrCreate()

sparks

In [21]:
# Lets create a simple Python decorator - {get_time} to get the execution timings
# If you dont know about Python decorators - check out : https://www.geeksforgeeks.org/decorators-in-python/
import time

def get_time(func):
    def inner_get_time() -> str:
        start_time = time.time()
        func()
        end_time = time.time()
        return (f"Execution time: {(end_time - start_time)*1000} ms")
    print(inner_get_time())

In [52]:
# Set up to read from JDBC SQLite database
driver: str = "org.sqlite.JDBC"
db_path: str = "dataset/jdbc/demo-sqlite.db"
jdbc_url: str = "jdbc:sqlite:" + db_path
table_name: str = "sales_csv"

In [63]:
# Lets read the SQLite table using JDBC driver and validate the data
df = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load()

df.printSchema()
df.show(10, False)

root
 |-- transacted_at: string (nullable = true)
 |-- trx_id: integer (nullable = true)
 |-- retailer_id: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- city_id: integer (nullable = true)

+------------------------+----------+-----------+---------------------------------------------+-------+----------+
|transacted_at           |trx_id    |retailer_id|description                                  |amount |city_id   |
+------------------------+----------+-----------+---------------------------------------------+-------+----------+
|2017-11-24T19:00:00.000Z|1995601912|2077350195 |Walgreen       11-25                         |197.23 |216510442 |
|2017-11-24T19:00:00.000Z|1734117021|644879053  |unkn    ppd id: 768641     11-26             |8.58   |930259917 |
|2017-11-24T19:00:00.000Z|1734117022|847200066  |Wal-Mart  ppd id: 555914     Algiers    11-26|1737.26|1646415505|
|2017-11-24T19:00:00.000Z|1734117030|1953761884 |Home D

In [65]:
# Checking the explain plan
df.explain(True)

== Parsed Logical Plan ==
Relation [transacted_at#414,trx_id#415,retailer_id#416,description#417,amount#418,city_id#419] JDBCRelation(sales_csv) [numPartitions=1]

== Analyzed Logical Plan ==
transacted_at: string, trx_id: int, retailer_id: int, description: string, amount: double, city_id: int
Relation [transacted_at#414,trx_id#415,retailer_id#416,description#417,amount#418,city_id#419] JDBCRelation(sales_csv) [numPartitions=1]

== Optimized Logical Plan ==
Relation [transacted_at#414,trx_id#415,retailer_id#416,description#417,amount#418,city_id#419] JDBCRelation(sales_csv) [numPartitions=1]

== Physical Plan ==
*(1) Scan JDBCRelation(sales_csv) [numPartitions=1] [transacted_at#414,trx_id#415,retailer_id#416,description#417,amount#418,city_id#419] PushedFilters: [], ReadSchema: struct<transacted_at:string,trx_id:int,retailer_id:int,description:string,amount:double,city_id:int>



In [80]:
# Checking the performance for Full read without any Predicate Pushdown
@get_time
def x():
    df_full = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load()
    
    df_full.write.format("noop").mode("overwrite").save()
    df_full.explain(True)

== Parsed Logical Plan ==
Relation [transacted_at#1131,trx_id#1132,retailer_id#1133,description#1134,amount#1135,city_id#1136] JDBCRelation(sales_csv) [numPartitions=1]

== Analyzed Logical Plan ==
transacted_at: string, trx_id: int, retailer_id: int, description: string, amount: double, city_id: int
Relation [transacted_at#1131,trx_id#1132,retailer_id#1133,description#1134,amount#1135,city_id#1136] JDBCRelation(sales_csv) [numPartitions=1]

== Optimized Logical Plan ==
Relation [transacted_at#1131,trx_id#1132,retailer_id#1133,description#1134,amount#1135,city_id#1136] JDBCRelation(sales_csv) [numPartitions=1]

== Physical Plan ==
*(1) Scan JDBCRelation(sales_csv) [numPartitions=1] [transacted_at#1131,trx_id#1132,retailer_id#1133,description#1134,amount#1135,city_id#1136] PushedFilters: [], ReadSchema: struct<transacted_at:string,trx_id:int,retailer_id:int,description:string,amount:double,city_id:int>

Execution time: 9063.788414001465 ms


In [71]:
# Checking the performance for Predicate Pushdown
@get_time
def x():
    df_filtered = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load() \
    .filter("city_id = 216510442")
    
    df_filtered.write.format("noop").mode("overwrite").save()
    df_filtered.explain(True)

== Parsed Logical Plan ==
'Filter ('city_id = 216510442)
+- Relation [transacted_at#541,trx_id#542,retailer_id#543,description#544,amount#545,city_id#546] JDBCRelation(sales_csv) [numPartitions=1]

== Analyzed Logical Plan ==
transacted_at: string, trx_id: int, retailer_id: int, description: string, amount: double, city_id: int
Filter (city_id#546 = 216510442)
+- Relation [transacted_at#541,trx_id#542,retailer_id#543,description#544,amount#545,city_id#546] JDBCRelation(sales_csv) [numPartitions=1]

== Optimized Logical Plan ==
Filter (isnotnull(city_id#546) AND (city_id#546 = 216510442))
+- Relation [transacted_at#541,trx_id#542,retailer_id#543,description#544,amount#545,city_id#546] JDBCRelation(sales_csv) [numPartitions=1]

== Physical Plan ==
*(1) Scan JDBCRelation(sales_csv) [numPartitions=1] [transacted_at#541,trx_id#542,retailer_id#543,description#544,amount#545,city_id#546] PushedFilters: [*IsNotNull(city_id), *EqualTo(city_id,216510442)], ReadSchema: struct<transacted_at:string

In [73]:
# Cripple the performance for Predicate Pushdown
@get_time
def x():
    df_filtered = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load() \
    .cache() \
    .filter("city_id = 216510442")
    
    df_filtered.write.format("noop").mode("overwrite").save()
    df_filtered.explain(True)

== Parsed Logical Plan ==
'Filter ('city_id = 216510442)
+- Relation [transacted_at#817,trx_id#818,retailer_id#819,description#820,amount#821,city_id#822] JDBCRelation(sales_csv) [numPartitions=1]

== Analyzed Logical Plan ==
transacted_at: string, trx_id: int, retailer_id: int, description: string, amount: double, city_id: int
Filter (city_id#822 = 216510442)
+- Relation [transacted_at#817,trx_id#818,retailer_id#819,description#820,amount#821,city_id#822] JDBCRelation(sales_csv) [numPartitions=1]

== Optimized Logical Plan ==
Filter (isnotnull(city_id#822) AND (city_id#822 = 216510442))
+- InMemoryRelation [transacted_at#817, trx_id#818, retailer_id#819, description#820, amount#821, city_id#822], StorageLevel(disk, memory, deserialized, 1 replicas)
      +- *(1) Scan JDBCRelation(sales_csv) [numPartitions=1] [transacted_at#817,trx_id#818,retailer_id#819,description#820,amount#821,city_id#822] PushedFilters: [], ReadSchema: struct<transacted_at:string,trx_id:int,retailer_id:int,descrip

In [79]:
# We can even push down Queries for perfomance benifits
pushDownQuery = """(select city_id, count(1) as cnt from sales_csv group by city_id) as sales_csv"""
@get_time
def x():
    df_filtered = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", pushDownQuery) \
    .load()
    
    df_filtered.write.format("noop").mode("overwrite").save()
    df_filtered.explain(True)

== Parsed Logical Plan ==
Relation [city_id#1125,cnt#1126] JDBCRelation((select city_id, count(1) as cnt from sales_csv group by city_id) as sales_csv) [numPartitions=1]

== Analyzed Logical Plan ==
city_id: int, cnt: decimal(38,18)
Relation [city_id#1125,cnt#1126] JDBCRelation((select city_id, count(1) as cnt from sales_csv group by city_id) as sales_csv) [numPartitions=1]

== Optimized Logical Plan ==
Relation [city_id#1125,cnt#1126] JDBCRelation((select city_id, count(1) as cnt from sales_csv group by city_id) as sales_csv) [numPartitions=1]

== Physical Plan ==
*(1) Scan JDBCRelation((select city_id, count(1) as cnt from sales_csv group by city_id) as sales_csv) [numPartitions=1] [city_id#1125,cnt#1126] PushedFilters: [], ReadSchema: struct<city_id:int,cnt:decimal(38,18)>

Execution time: 2027.7936458587646 ms
